<a href="https://colab.research.google.com/github/SarahLares/introduction_to_NLP/blob/master/Twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re    # for regular expressions 
import nltk  # for text manipulation 
import string 
import warnings 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt  

%matplotlib inline

In [0]:
url_test =  'https://raw.githubusercontent.com/SarahLares/introduction_to_NLP/master/test_tweets_anuFYb8.csv'
url_train =  'https://raw.githubusercontent.com/SarahLares/introduction_to_NLP/master/train_E6oV3lV.csv'

In [0]:
train  = pd.read_csv(url_train) 
test = pd.read_csv(url_test)

Let’s check dimensions of the train and test dataset.


In [20]:
train.shape, test.shape

((31962, 3), (17197, 2))

# Data Inspection

Let’s check out a few non racist/sexist tweets.

In [18]:
train.loc[train['label']==0].head(10)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


Now check out a few racist/sexist tweets.

In [19]:
train.loc[train['label']==1].head(10)

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...
56,57,1,@user lets fight against #love #peace
68,69,1,ð©the white establishment can't have blk fol...
77,78,1,"@user hey, white people: you can call people '..."
82,83,1,how the #altright uses &amp; insecurity to lu...
111,112,1,@user i'm not interested in a #linguistics tha...
